In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys


Mounted at /content/gdrive


In [ ]:
!pip install kornia
!pip install torch==1.11.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.4/899.4 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 2.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu121
    Uninstalling torch-2.5.1+cu121:
      Successfully uninstalled torch-2.5.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.13.2 requires torch>=1.13.0, but you have torch 1.11.0 which is incompatible.
torchaudio 2.5.1+cu121 requires torch==2.5.1, but you have torch 1.11.0 which is incompatible.
torchvision 0.20.1+cu121 requires torch==2.5.1, but you have torch 1.11.0 which is incompatible.


In [ ]:
import random
import readline
import numpy as np
import torch
import kornia


In [ ]:
HAS_CUDA = torch.cuda.is_available()
USE_CUDA = False
GRASS_INDEX = 'grass'
frame_h = 720
frame_w = 1280
template_h = 74
template_w = 115
output_image_size = (256, 256)


In [ ]:
def get_perspective_transform(src, dst):

        def ax(p, q):
            ones = torch.ones_like(p)[..., 0:1]
            zeros = torch.zeros_like(p)[..., 0:1]
            return torch.cat(
                [p[:, 0:1], p[:, 1:2], ones, zeros, zeros, zeros,
                -p[:, 0:1] * q[:, 0:1], -p[:, 1:2] * q[:, 0:1]
                ], dim=1)

        def ay(p, q):
            ones = torch.ones_like(p)[..., 0:1]
            zeros = torch.zeros_like(p)[..., 0:1]
            return torch.cat(
                [zeros, zeros, zeros, p[:, 0:1], p[:, 1:2], ones,
                -p[:, 0:1] * q[:, 1:2], -p[:, 1:2] * q[:, 1:2]], dim=1)

        p = []
        p.append(ax(src[:, 0], dst[:, 0]))
        p.append(ay(src[:, 0], dst[:, 0]))

        p.append(ax(src[:, 1], dst[:, 1]))
        p.append(ay(src[:, 1], dst[:, 1]))

        p.append(ax(src[:, 2], dst[:, 2]))
        p.append(ay(src[:, 2], dst[:, 2]))

        p.append(ax(src[:, 3], dst[:, 3]))
        p.append(ay(src[:, 3], dst[:, 3]))

        A = torch.stack(p, dim=1)

        b = torch.stack([
            dst[:, 0:1, 0], dst[:, 0:1, 1],
            dst[:, 1:2, 0], dst[:, 1:2, 1],
            dst[:, 2:3, 0], dst[:, 2:3, 1],
            dst[:, 3:4, 0], dst[:, 3:4, 1],
        ], dim=1)

        X, LU = torch.solve(b, A)

        batch_size = src.shape[0]
        M = torch.ones(batch_size, 9, device=src.device, dtype=src.dtype)
        M[..., :8] = torch.squeeze(X, dim=-1)
        return M.view(-1, 3, 3)  # Bx3x3

In [ ]:
def to_numpy(cuda_var):
    return cuda_var.data.cpu().numpy()


def to_torch(np_array):
    if True:
        tensor = torch.from_numpy(np_array).float().cuda()
    else:
        tensor = torch.from_numpy(np_array).float()
    return torch.autograd.Variable(tensor, requires_grad=False)

def FULL_CANON4PTS_NP():
    return np.array([[-0.5, -0.5], [-0.5, 0.5], [0.5, 0.5], [0.5, -0.5]], dtype=np.float32)

In [ ]:
def get_frame_space_scaling_homography():
        src_4pts = to_torch(FULL_CANON4PTS_NP())
        dest_4pts = to_torch(np.array([[0, 0], [0, frame_h], [frame_w, frame_h], [frame_w, 0]], dtype=np.float32))
        scaling_transformation = get_perspective_transform(src_4pts[None], dest_4pts[None])
        scaling_transformation = to_numpy(scaling_transformation[0])
        return scaling_transformation

def get_template_space_scaling_homography():
    src_4pts = to_torch(np.array([[0, 0], [0, template_h], [template_w, template_h], [template_w, 0]], dtype=np.float32))
    dest_4pts = to_torch(FULL_CANON4PTS_NP())
    scaling_transformation = get_perspective_transform(src_4pts[None], dest_4pts[None])
    scaling_transformation = to_numpy(scaling_transformation[0])
    return scaling_transformation


def get_homography_by_path(homo_mat):

    frame_scaling = get_frame_space_scaling_homography()
    homo_mat = np.matmul(homo_mat, frame_scaling)

    template_scaling = get_template_space_scaling_homography()
    homo_mat = np.matmul(template_scaling, homo_mat)
    homo_mat = homo_mat / homo_mat[2, 2]
    return homo_mat


In [ ]:
import os

In [ ]:
homo_dict = {}
homography_folder = '/content/gdrive/MyDrive/soccer_data/raw/test'
for fl in os.listdir(homography_folder):
    f = os.path.join(homography_folder, fl)
    if f[-3:] == "rix":
      data = np.loadtxt(f)
      modified_homography_data = get_homography_by_path(data)

      homo_dict[fl.split('.')[0]] = str(modified_homography_data)



<ipython-input-5-a747417966f5>:96: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the LU factorization see torch.lu, which can be used with torch.lu_solve or torch.lu_unpack.
X = torch.solve(B, A).solution
should be replaced with
X = torch.linalg.solve(A, B) (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:859.)
  X, LU = torch.solve(b, A)


In [ ]:
homo_dict

{'1': '[[ 0.24595309 -0.07008269 -0.27732284]\n [-0.1650696   1.26257125 -0.1380419 ]\n [ 0.06429313  1.39161385  1.        ]]',
 '10': '[[ 0.21101108  0.04484299  0.49661802]\n [ 0.20746508  1.22555315 -0.24905761]\n [-0.04652761  1.21849738  1.        ]]',
 '100': '[[ 0.20471276 -0.02423265 -0.3965206 ]\n [-0.11960113  1.17929352  0.11191389]\n [ 0.04373913  0.86593963  1.        ]]',
 '101': '[[ 0.21402202 -0.01505467 -0.44037204]\n [-0.14199291  1.23230626  0.06756938]\n [ 0.03840949  0.87766379  1.        ]]',
 '102': '[[ 3.40502776e-01 -2.72334075e-03 -1.45151447e-01]\n [-7.66566571e-02  1.69493649e+00  3.99350405e-02]\n [-2.59350920e-04  1.19390505e+00  1.00000000e+00]]',
 '103': '[[ 0.27869972 -0.02870445  0.41374631]\n [ 0.14125562  1.4497091  -0.05839414]\n [-0.00185972  0.94421921  1.        ]]',
 '104': '[[ 2.51390734e-01  2.32240867e-02  4.52912707e-01]\n [ 1.57581232e-01  1.46259853e+00  9.83249407e-05]\n [-5.83106152e-02  1.07885555e+00  1.00000000e+00]]',
 '105': '[[ 0.

In [ ]:
import json
with open('/content/gdrive/MyDrive/soccer_data/raw/modified_test_homo_data.json', 'w') as fp:
    json.dump(homo_dict, fp)